In [192]:
import pandas as pd
import numpy as np
import liwc
import re
from collections import Counter,OrderedDict
import csv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [193]:
data = pd.read_csv('./data/hrdata_OSF.csv')

focal_cats = np.array(['Achiev', 'Adverbs', 'Affect', 'Anger', 'Anx', 'Article',
               'Assent', 'AuxVb', 'Body', 'Cause', 'Certain',
               'Conj', 'Death', 'Discrep', 'Excl', 'Family',
               'Feel', 'Filler', 'Friends', 'Future', 'Health',
               'Hear', 'Home', 'Humans', 'I', 'Incl', 'Ingest', 'Inhib',
               'Insight', 'Ipron', 'Leisure', 'Money', 'Motion', 'Negate',
               'Negemo', 'Nonflu', 'Numbers', 'Past', 'Percept', 'Posemo',
               'Prep', 'Present', 'Quant',
               'Relig', 'Sad', 'See', 'Sexual', 'SheHe', 'Social', 'Space',
               'Swear', 'Tentat', 'They', 'Time', 'We', 'Work', 'You'])

email = pd.read_csv('./data/SMS.csv',encoding='cp1252')

parse, category_names = liwc.load_token_parser('LIWC2007_English100131.dic')

In [194]:
m1_emp = []
m2_emp = []

t_m1 = 5
t_m2 = 17

for d in data.to_dict('record'):
    bank = d['bank']
    month = d['month']
    name = d['name']

    if bank != 'T' and month>= t_m1-3 and month<= t_m1:
        m1_emp.append({'name':name, 'bank': bank})

    if bank != 'L' and month>= t_m2-3 and month<= t_m2:
        m2_emp.append({'name':name, 'bank': bank})

/var/folders/tj/yk7dvkjd2_ndkr_0hlr4cgq80000gn/T/ipykernel_87285/1397543421.py:7: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  for d in data.to_dict('record'):


In [195]:
# unique
m1_emp = list({v['name']:v for v in m1_emp}.values())

m2_emp = list({v['name']:v for v in m2_emp}.values())

In [196]:
def tokenize(text):
    # you may want to use a smarter tokenizer
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0)


In [197]:
df = {}
volume = {}
for e in email.to_dict('records'):
    message = e['Message_body'].lower()
    tokens = tokenize(message)
    counts = Counter(category for token in tokens for category in parse(token))
    
    month = e['month']
    sender = e['sender']
    bank = e['sender'][0:1]
    receivers = e['receivers']

    temp = {}
    for c in dict(counts):
        strs = c[0].upper() + c[1:]
        if strs in focal_cats:
            temp[c] = counts[c]

    for cata in focal_cats:
        if cata.lower() not in temp:
            temp[cata] = 0

    if sender in volume:
        num = volume[sender] + 1
        volume[sender] = num
    else:
        volume[sender] = 1
    
    to_L = 0
    to_T = 0
    to_B = 0

    if sender[0]=='L':
        to_L = 1
    elif sender[0]=='T':
        to_T = 1
    else:
        to_B = 1


    df[sender]={
        'month':month,
        'sender':sender,
        'receivers':receivers,
        'bank': bank,
        'to_L':to_L,
        'to_T':to_T,
        'to_B':to_B
        }
    temp = OrderedDict(sorted(temp.items()))
    for cata in temp:
        c = cata.capitalize()
        df[sender][c] = temp[cata]
    # print(dict(counts))

In [198]:
for d in df:
    df[d]['n.terms'] = volume[d]

In [199]:
csvlist = []

for d in df:
    csvlist.append(df[d])

In [200]:
csvlist

[{'month': 1,
  'sender': 'B0004',
  'receivers': 'B0703',
  'bank': 'B',
  'to_L': 0,
  'to_T': 0,
  'to_B': 1,
  'Achiev': 0,
  'Adverbs': 0,
  'Anger': 0,
  'Anx': 0,
  'Article': 0,
  'Assent': 0,
  'Auxvb': 0,
  'Body': 0,
  'Certain': 0,
  'Conj': 0,
  'Death': 0,
  'Discrep': 0,
  'Excl': 0,
  'Family': 0,
  'Feel': 0,
  'Filler': 0,
  'Friends': 0,
  'Future': 0,
  'Health': 0,
  'Hear': 0,
  'Home': 0,
  'I': 0,
  'Incl': 0,
  'Ingest': 0,
  'Inhib': 0,
  'Insight': 0,
  'Ipron': 0,
  'Leisure': 0,
  'Money': 0,
  'Motion': 0,
  'Negate': 0,
  'Negemo': 0,
  'Nonflu': 0,
  'Numbers': 0,
  'Past': 0,
  'Percept': 0,
  'Prep': 0,
  'Quant': 0,
  'Relig': 0,
  'Sad': 0,
  'See': 0,
  'Sexual': 0,
  'Shehe': 0,
  'Space': 0,
  'Swear': 0,
  'Tentat': 0,
  'They': 0,
  'We': 0,
  'Work': 0,
  'Affect': 2,
  'Cause': 1,
  'Humans': 1,
  'Posemo': 2,
  'Present': 1,
  'Social': 2,
  'Time': 1,
  'You': 1,
  'n.terms': 2},
 {'month': 1,
  'sender': 'B0071',
  'receivers': 'B0733',
  '

In [201]:
# columns= ['month','sender','receivers','bank','to_B','to_L','to_T','Achiev','Adverbs','Anger','Anx','Article','Assent','AuxVb',
#   'Body','Certain','Conj','Death','Discrep','Excl','Family','Feel','Filler','Friends','Future',
#   'Health','Hear','Home','I','Incl','Ingest','Inhib','Insight','Ipron','Leisure','Money','Motion',
#   'Negate','Negemo','Nonflu','Numbers','Past','Percept','Prep','Quant','Relig','Sad','See','Sexual',
#   'SheHe','Space','Swear','Tentat','They','We','Work','Affect','Cause','Humans','Posemo','Present','Social'
#   ,'Time','You','n.terms']

# with open('1.csv', 'w') as csvfile:
#     w = csv.writer(csvfile)
#     w.writerow(columns)
#     for d in csvlist:
#         w.writerow(d.values())

In [202]:
# filter
# to_B + to_L + to_T != 0 and sender in m1 empl
m1_emp_dict = []
for emp in m1_emp:
    m1_emp_dict.append(emp['name'])

X = []

for data in csvlist:
    if data['sender'] in m1_emp_dict and (int(data['to_L'])+int(data['to_B'])+int(data['to_T'])!=0):
        data.pop('to_L')
        data.pop('to_B')
        data.pop('to_T')
        data.pop('receivers')
        X.append(data)

X = pd.DataFrame(X)
y = pd.DataFrame(X['bank']);
X = X.drop(columns=['bank','sender','month'],axis=1);

In [203]:
# divide 50/50 train and test
train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train

In [205]:
# model 1
# monthly iterate through



In [210]:
y_test.head(10)

,bank
110,B
419,L
564,B
77,B
181,B
284,B
10,B
469,L
78,B
349,L


In [211]:
# model 2
# a month m1=5 m2=17
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train,y_train.values.ravel())

clf.predict(X_test)

array(['B', 'B', 'B', 'L', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'L', 'B', 'L', 'B', 'L', 'B', 'L', 'B', 'L', 'L', 'B', 'B',
       'B', 'B', 'L', 'B', 'B', 'L', 'B', 'B', 'B', 'B', 'B', 'L', 'L',
       'B', 'B', 'B', 'B', 'L', 'B', 'B', 'B', 'B', 'B', 'L', 'B', 'B',
       'B', 'L', 'L', 'B', 'B', 'L', 'B', 'B', 'B', 'B', 'L', 'B', 'B',
       'B', 'L', 'L', 'B', 'L', 'B', 'B', 'L', 'B', 'B', 'L', 'B', 'B',
       'L', 'L', 'B', 'L', 'L', 'B', 'L', 'L', 'B', 'B', 'L', 'B', 'L',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'L', 'B', 'B', 'B', 'L', 'B',
       'B', 'B', 'B', 'B', 'B', 'L', 'B', 'B', 'L', 'L', 'B', 'B', 'L',
       'L', 'L', 'B', 'B', 'B', 'B', 'L', 'B', 'B', 'L', 'B', 'B', 'B',
       'B', 'L', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'L', 'B', 'B', 'B', 'B', 'B', 'L', 'L', 'L', 'B', 'L', 'B',
       'B', 'B', 'B', 'L', 'L', 'B', 'B', 'L', 'B', 'B', 'B', 'B', 'L',
       'L', 'B', 'B', 'L', 'L', 'B', 'B', 'L', 'B', 'L', 'B', 'L